# CREMA-D Emotion Recognition benchmark model using MFCC/FBANK features

In this notebook, we will train different classifiers on ER on the CREMA-D dataset using MFCC/FBANK features.

Moreover, we provide the performance that can acquired by classifying samples by chance.

### Import packages

In [21]:
import os

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import plot_confusion_matrix, classification_report, balanced_accuracy_score, f1_score, precision_score, recall_score

#### Training data paths

### Define paths

In [2]:
mfcc_train_dir = os.path.join("/media", "datastore", "c-matsty-data", "CREMA-D_Preproc_1_training_MFCC")
mfcc_train_input_path = os.path.join(mfcc_train_dir, "input_data")
mfcc_train_labels_path = os.path.join(mfcc_train_dir, "labels")
mfcc_train_actors_path = os.path.join(mfcc_train_dir, "actors")
fbank_train_dir = os.path.join("/media", "datastore", "c-matsty-data", "CREMA-D_Preproc_1_training_FBANK")
fbank_train_input_path = os.path.join(fbank_train_dir, "input_data")
fbank_train_labels_path = os.path.join(fbank_train_dir, "labels")
fbank_train_actors_path = os.path.join(fbank_train_dir, "actors")

#### Test data paths

In [3]:
mfcc_test_dir = os.path.join("/media", "datastore", "c-matsty-data", "CREMA-D_Preproc_1_test_MFCC")
mfcc_test_input_path = os.path.join(mfcc_test_dir, "input_data")
mfcc_test_labels_path = os.path.join(mfcc_test_dir, "labels")
mfcc_test_actors_path = os.path.join(mfcc_test_dir, "actors")
fbank_test_dir = os.path.join("/media", "datastore", "c-matsty-data", "CREMA-D_Preproc_1_test_FBANK")
fbank_test_input_path = os.path.join(fbank_test_dir, "input_data")
fbank_test_labels_path = os.path.join(fbank_test_dir, "labels")
fbank_test_actors_path = os.path.join(fbank_test_dir, "actors")

### Load data

In [63]:
def load_data(input_path, labels_path, actors_path):
    X = []
    Y = []
    A = []
    for input_file_name in os.listdir(input_path):
        file_name = input_file_name.split(".")[0]
        x = np.load(os.path.join(input_path, input_file_name))
        y = np.load(os.path.join(labels_path, file_name + "_labels.npy"))[..., np.newaxis]
        actors = np.load(os.path.join(actors_path, file_name + "_actors.npy"))[..., np.newaxis]
        X.append(x)
        Y.append(y)
        A.append(actors)
    X = np.vstack(X)
    Y = np.vstack(Y)
    A = np.vstack(A)
    return X, Y, A

###### Load training data

In [ ]:
X_mfcc_tr, y_mfcc_tr, actors_mfcc_tr = load_data(mfcc_train_input_path, mfcc_train_labels_path, mfcc_train_actors_path)

In [ ]:
X_fbank_tr, y_fbank_tr, actors_fbank_tr = load_data(fbank_train_input_path, fbank_train_labels_path, fbank_train_actors_path)

###### Load test data

In [ ]:
X_mfcc_ts, y_mfcc_ts, actors_mfcc_ts = load_data(mfcc_test_input_path, mfcc_test_labels_path, mfcc_test_actors_path)

In [ ]:
X_fbank_ts, y_fbank_ts, actors_fbank_ts = load_data(fbank_test_input_path, fbank_test_labels_path, fbank_test_actors_path)

In [ ]:
label_dict = {value: index  for index, value in enumerate(np.unique(y_mfcc_tr))}

### Prepare data for training

#### Turn labels from strings to integer identifiers

In [51]:
y_mfcc_tr = np.vectorize(label_dict.get)(y_mfcc_tr)
y_fbank_tr = np.vectorize(label_dict.get)(y_fbank_tr)
y_mfcc_ts = np.vectorize(label_dict.get)(y_mfcc_ts)
y_fbank_ts = np.vectorize(label_dict.get)(y_fbank_ts)

#### Reshape input arrays and labels

In [52]:
X_mfcc_tr = X_mfcc_tr.reshape((X_mfcc_tr.shape[0], X_mfcc_tr.shape[1] * X_mfcc_tr.shape[2]))
X_mfcc_ts = X_mfcc_ts.reshape((X_mfcc_ts.shape[0], X_mfcc_ts.shape[1] * X_mfcc_ts.shape[2]))
X_fbank_tr = X_fbank_tr.reshape((X_fbank_tr.shape[0], X_fbank_tr.shape[1] * X_fbank_tr.shape[2]))
X_fbank_ts = X_fbank_ts.reshape((X_fbank_ts.shape[0], X_fbank_ts.shape[1] * X_fbank_ts.shape[2]))

In [53]:
y_mfcc_tr = y_mfcc_tr.flatten()
y_mfcc_ts = y_mfcc_ts.flatten()
y_fbank_tr = y_fbank_tr.flatten()
y_fbank_ts = y_fbank_ts.flatten()

### Normalize data

In [54]:
mfcc_mean = X_mfcc_tr.mean()
mfcc_std = X_mfcc_tr.std()
X_mfcc_tr = (X_mfcc_tr - mfcc_mean) / mfcc_std 
X_mfcc_ts = (X_mfcc_ts - mfcc_mean) / mfcc_std

In [55]:
fbank_mean = X_fbank_tr.mean()
fbank_std = X_fbank_tr.std()
X_fbank_tr = (X_fbank_tr - fbank_mean) / fbank_std 
X_fbank_ts = (X_fbank_ts - fbank_mean) / fbank_std

### Define sample weights

In [56]:
class_counts = [len(y_mfcc_tr[y_mfcc_tr == i]) for i in label_dict.values()]
class_weights = [max(class_counts)/class_count for class_count in class_counts]
class_weight_dict = {class_idx: class_weight for class_idx, class_weight in zip(label_dict.values(), class_weights)}

In [57]:
label_dict

{'eight': 0,
 'five': 1,
 'four': 2,
 'nine': 3,
 'one': 4,
 'seven': 5,
 'six': 6,
 'three': 7,
 'two': 8,
 'zero': 9}

### Train and test logistic regression classifier

##### Train with MFCC features

In [58]:
multi_class = 'multinomial'
model = LogisticRegression(multi_class=multi_class, max_iter=15000, random_state=0)

In [59]:
sample_weight = [class_weight_dict[label] for label in y_mfcc_tr]

In [ ]:
model = model.fit(X_mfcc_tr, y_mfcc_tr, sample_weight=sample_weight)

In [ ]:
y_preds = model.predict(X_mfcc_ts)
print(classification_report(y_mfcc_ts, y_preds))
print(balanced_accuracy_score(y_mfcc_ts, y_preds))

##### Train with FBANK features

In [ ]:
model = LogisticRegression(multi_class=multi_class, max_iter=15000, random_state=0)

In [ ]:
model = model.fit(X_fbank_tr, y_fbank_tr, sample_weight=sample_weight)

In [ ]:
y_preds = model.predict(X_fbank_ts)
print(classification_report(y_fbank_ts, y_preds))
print(balanced_accuracy_score(y_fbank_ts, y_preds))

### Train and test RandomForest classifier

##### Train with MFCC features

In [ ]:
model = RandomForestClassifier(n_estimators=400, random_state=0)

In [ ]:
model = model.fit(X_mfcc_tr, y_mfcc_tr, sample_weight=sample_weight)

In [ ]:
y_preds = model.predict(X_mfcc_ts)
print(classification_report(y_mfcc_ts, y_preds))
print(balanced_accuracy_score(y_mfcc_ts, y_preds))

##### Train with FBANK features

In [ ]:
model = RandomForestClassifier(n_estimators=400, random_state=0)

In [ ]:
model = model.fit(X_fbank_tr, y_fbank_tr, sample_weight=sample_weight)

In [ ]:
y_preds = model.predict(X_fbank_ts)
print(classification_report(y_fbank_ts, y_preds))
print(balanced_accuracy_score(y_fbank_ts, y_preds))

### Chance level performance

In [ ]:
trials = 100
avg_f1 = 0.0
avg_prec = 0.0
avg_rec = 0.0
for i in range(trials):
    y_preds = np.random.choice(np.unique(y_fbank_tr), y_fbank_ts.shape[0])
    avg_f1 += f1_score(y_fbank_ts, y_preds, average='macro')
    avg_prec += precision_score(y_fbank_ts, y_preds, average='macro')
    avg_rec += recall_score(y_fbank_ts, y_preds, average='macro')
avg_f1 /= trials
avg_prec /= trials
avg_rec /= trials
print("Macro f1: {}".format(avg_f1))
print("Macro recall: {}".format(avg_rec))
print("Macro precision: {}".format(avg_prec))